### For selected ClearCut AOI prepare Planet and Sentinel2 images. Show clearcuts on AOI. Compare quality

In [1]:
import os
import rasterio
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio.mask
import tempfile
import shapely
import re

from rasterio.features import rasterize, shapes
from rasterio.merge import merge
from shapely.geometry import Polygon, shape, box
from pathlib import Path
from datetime import datetime, timedelta

from sentinel2download.downloader import Sentinel2Downloader

In [2]:
BASE = f"/home/{os.getenv('NB_USER')}/work"

WORKDIR = os.path.join(BASE, "notebooks/example/planet_vs_sentinel")
IMGDIR = os.path.join(BASE, "satellite_imagery")
RESULTS_DIR = os.path.join(BASE, "results/example/planet_vs_sentinel")

### Read AOI

In [3]:
def read_aoi(path, crs="epsg:4326"):
    df = gpd.read_file(path) 
    if str(df.crs) != crs:
        print(f"{path}: {df.crs}. Transform to {crs}")
    df.to_crs(crs, inplace=True)
    return df

In [5]:
aoi_path = os.getenv("AOI", os.path.join(WORKDIR, "aoi/Horodnie_20190801_20190831.geojson")) # Pervukhynka_20190801_20190831.geojson 

aoi = read_aoi(aoi_path)

In [6]:
aoi

,id,geometry
0,1,"MULTIPOLYGON (((35.27794 50.15967, 35.22820 50..."


In [7]:
start_date = '2019-08-01'
end_date = '2019-08-31'

In [8]:
def filter_planet(grid, start_date, end_date):
    # grid.acquired = pd.to_datetime(grid.acquired)
    grid = grid.loc[(grid.ground_control == "1") & 
                    (grid.item_type == "PSScene4Band") &
                    (grid.quality_category == "standard") &
                    (grid.cloud_cover == 0) &
                    (grid.acquired >= start_date) &
                    (grid.acquired <= end_date) &
                    (grid.snow_ice_percent == 0) &
                    (grid.clear_percent >= 95)].reset_index(drop=True)
    return grid

In [9]:
def epsg_code(longitude, latitude):
    """
    Generates EPSG code from lon, lat
    :param longitude: float
    :param latitude: float
    :return: int, EPSG code
    """

    def _zone_number(lat, lon):
        if 56 <= lat < 64 and 3 <= lon < 12:
            return 32
        if 72 <= lat <= 84 and lon >= 0:
            if lon < 9:
                return 31
            elif lon < 21:
                return 33
            elif lon < 33:
                return 35
            elif lon < 42:
                return 37

        return int((lon + 180) / 6) + 1

    zone = _zone_number(latitude, longitude)

    if latitude > 0:
        return 32600 + zone
    else:
        return 32700 + zone

In [10]:
def _intersect(aoi, grid, limit=1):
    """
    Find all tiles that intersects given region with area >= limit km2
    :param limit: float, min intersection area in km2
    :return: (GeoDataFrame, epsg), precised intersected tiles and UTM zone code
    """

    
    # Get the indices of the tiles that are likely to be inside the bounding box of the given Polygon
    geometry = aoi.geometry[0]

    tiles_indexes = list(grid.sindex.intersection(geometry.bounds))
    intersected_grid = grid.loc[tiles_indexes]
    
    # print(intersected_grid)

    # Make the precise tiles in Polygon query
    intersected_grid = intersected_grid.loc[grid.intersects(geometry)]

    # intersection area
    epsg = epsg_code(geometry.centroid.x, geometry.centroid.y)

    # to UTM projection in meters
    aoi.to_crs(epsg=epsg, inplace=True)
    intersected_grid.to_crs(epsg=epsg, inplace=True)

    return intersected_grid, epsg

In [11]:
def _overlap_aoi_row(aoi, bbox_intersected, crs):
    
    result = list()

    intersected = bbox_intersected.copy()
    rest_aoi = gpd.GeoDataFrame([aoi], crs=crs)
    
    
    try:
        while rest_aoi.area.sum() > 0:
       
            res_intersection = gpd.overlay(rest_aoi, intersected, how="intersection") # intersection area with tiles
            biggest_area_id = res_intersection.area.argmax() # max intersected area
            # print("res_intersection")
            # print(res_intersection)

            tile_id = res_intersection.loc[biggest_area_id, "Name"]
            intersected_aoi = res_intersection.loc[biggest_area_id, "geometry"]
        
            # print(tile_id)
        
            result.append({"Name": tile_id, "geometry": intersected_aoi}) # store max intersected aoi and tile geometry
            biggest_tile = intersected.loc[intersected.Name == tile_id]

            rest_aoi = gpd.overlay(rest_aoi, biggest_tile, how="difference") # aoi minus biggest tile
            intersected = intersected[intersected.Name != tile_id] # remove used tile
    except Exception as ex:
        print(f"Error: {str(ex)}")
        
    return result

In [12]:
def overlap(aoi, grid):
    # first, find bbox intersection
    bbox = box(*aoi.total_bounds)
    bbox = gpd.GeoDataFrame(geometry=[box(*aoi.total_bounds)], crs=aoi.crs)
    
    bbox_intersected, epsg = _intersect(bbox, grid)
    # print(bbox_intersected)
    
    # precise intersection
    projected_aoi = aoi.copy().to_crs(epsg=epsg)
    
    results = list()
    for row in projected_aoi.itertuples():
        result = _overlap_aoi_row(row, bbox_intersected, projected_aoi.crs)
        results.extend(result)
        
    return gpd.GeoDataFrame(results, crs=epsg)

### 1. Planet images

In [13]:
planet_grid = gpd.read_file(os.path.join(BASE, "grids/planet_grid.geojson"), driver="GeoJSON")
planet_grid.head(1)

,id,acquired,anomalous_pixels,black_fill,cloud_cover,columns,epsg_code,grid_cell,ground_control,gsd,...,clear_percent,cloud_percent,heavy_haze_percent,light_haze_percent,shadow_percent,snow_ice_percent,visible_confidence_percent,visible_percent,year,geometry
0,1336590_3761711_2018-04-08_1021,2018-04-08T07:51:44,0.01,0.34,0.006,8000,32637,3761711,1,3.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,"POLYGON ((37.68174 48.95690, 37.67582 49.18171..."


In [14]:
planet_grid = filter_planet(planet_grid, start_date, end_date)

In [15]:
planet_grid = planet_grid[['id', 'acquired', 'geometry']].copy()
planet_grid.rename(columns={"id": "Name"}, inplace=True)

In [16]:
planet_grid.tail(3)

,Name,acquired,geometry
82,20190830_081708_1021,2019-08-30T08:17:08,"POLYGON ((35.60376 49.99066, 35.25423 50.04774..."
83,20190830_081709_1021,2019-08-30T08:17:09,"POLYGON ((35.57930 49.92516, 35.22936 49.98234..."
84,20190830_081710_1021,2019-08-30T08:17:10,"POLYGON ((35.55415 49.86005, 35.20563 49.91706..."


In [17]:
planet_tiles = overlap(aoi, planet_grid)

In [18]:
planet_tiles

,Name,geometry
0,20190826_081658_0f35,"POLYGON ((662704.134 5558867.810, 665939.948 5..."


In [19]:
planet_tiles = planet_tiles.merge(planet_grid[['Name', 'acquired']], on='Name')

In [20]:
planet_tiles

,Name,geometry,acquired
0,20190826_081658_0f35,"POLYGON ((662704.134 5558867.810, 665939.948 5...",2019-08-26T08:16:58


In [21]:
planet_tiles['date'] = planet_tiles.acquired.apply(lambda date:
                                                   datetime.strptime(date, '%Y-%m-%dT%H:%M:%S').strftime('%Y-%m-%d'))

In [22]:
planet_tiles

,Name,geometry,acquired,date
0,20190826_081658_0f35,"POLYGON ((662704.134 5558867.810, 665939.948 5...",2019-08-26T08:16:58,2019-08-26


In [23]:
def crop(input_path, output_path, polygon, name=None, date=None):
    with rasterio.open(input_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, [polygon], crop=True)
        out_meta = src.meta
        
        
        out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform,
                 "nodata": 0,
                        })
        #if out_image.shape[0] > 3:
         #   print(f"Image has {out_image.shape} bands. Extracting only first three TCI bands")
          #  out_image = out_image[:3, :, :]
            
           # out_meta.update({
            #'count': 3, })
        

    with rasterio.open(output_path, "w", **out_meta) as dest:
        if name:
            dest.update_tags(name=name)
        if date:
            dest.update_tags(start_date=date, end_date=date)
        dest.write(out_image)

In [24]:
def create_mosaic(files_to_mosaic, results_dir, name, start_date=None, end_date=None):
    src_files_to_mosaic = list()
    for fp in files_to_mosaic:
        src = rasterio.open(fp)
        src_files_to_mosaic.append(src)
    
    # crs of first input will be used
    mosaic, out_trans = merge(src_files_to_mosaic)
    out_meta = src.meta.copy()

    # Update the metadata
    out_meta.update({"driver": "GTiff",
                     "height": mosaic.shape[1],
                     "width": mosaic.shape[2],
                     "transform": out_trans,
                     })
    
    out_fp = os.path.join(results_dir, f"{name}_mosaic.tif.temp") 
    # print(out_fp)
    os.makedirs(os.path.dirname(out_fp), exist_ok=True)
    
    with rasterio.open(out_fp, "w", **out_meta) as dest:
        if name:
            dest.update_tags(name=name)
        if start_date:
            dest.update_tags(start_date=start_date)
        if end_date:
            dest.update_tags(end_date=end_date)
        dest.write(mosaic)
        
    os.rename(out_fp, out_fp[:-5])

### Prepair Planet images for AOI

In [25]:
with tempfile.TemporaryDirectory(dir=WORKDIR) as tmpdirname:
    print('Сreated temporary directory for calculations:', tmpdirname)
    
    files_to_mosaic = list()
    
    aoi_name = Path(aoi_path).stem
    for row in planet_tiles.itertuples():
        
        name = f"{aoi_name}_{row.Name}"
        
        input_path = os.path.join(IMGDIR, f"Planet/PSScene4Band/{row.Name}_3B_AnalyticMS.tif")
        output_path = os.path.join(WORKDIR, f"{name}_cropped.tif")
        
        crop(input_path, output_path, row.geometry, name, row.date)
        files_to_mosaic.append(output_path)
    
    # create_mosaic(files_to_mosaic, WORKDIR, "planet", planet_tiles.date.min(), planet_tiles.date.max())
    
    print(f"Calculations finished")

Сreated temporary directory for calculations: /home/jovyan/work/notebooks/example/planet_vs_sentinel/tmp9vkllwaw
Calculations finished


### 2. Sentinel2 images

In [26]:
sentinel_grid_path = os.path.join(BASE, "grids/sentinel2grid.geojson")

In [27]:
sentinel_grid = gpd.read_file(sentinel_grid_path)
sentinel_grid.head()

,Name,geometry
0,01CCV,"POLYGON Z ((180.000 -73.060 0.000, 176.865 -72..."
1,01CCV,"POLYGON Z ((-180.000 -72.073 0.000, -179.627 -..."
2,01CDH,"POLYGON Z ((180.000 -83.809 0.000, 174.713 -83..."
3,01CDH,"POLYGON Z ((-180.000 -82.826 0.000, -176.297 -..."
4,01CDJ,"POLYGON Z ((180.000 -82.913 0.000, 175.748 -82..."


In [28]:
sentinel_tiles = overlap(aoi, sentinel_grid)
sentinel_tiles 

,Name,geometry
0,36UXA,"POLYGON Z ((662704.134 5558867.810 0.000, 6659..."


In [29]:
def shift_dates(start_date, end_date, delta=2, format='%Y-%m-%d'):
    start = datetime.strptime(start_date, format)
    end = datetime.strptime(end_date, format)
    
    start = start - timedelta(days=delta)
    end = end + timedelta(days=delta)
    
    return datetime.strftime(start, format), datetime.strftime(end, format)

In [30]:
start, end = shift_dates(planet_tiles.date.min(), planet_tiles.date.max()) 
start, end

('2019-08-24', '2019-08-28')

#### Load Sentinel2 images

In [31]:
API_KEY = os.path.join(BASE, ".secret/sentinel2_google_api_key.json")
LOAD_DIR = os.path.join(BASE, "satellite_imagery")


BANDS = {'TCI', }
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 10.0, 'CLOUDY_PIXEL_PERCENTAGE': 5.0, }
PRODUCT_TYPE = 'L2A'

In [32]:
def load_images(api_key, tiles, start_date, end_date, output_dir, product_type="L2A"):
    loader = Sentinel2Downloader(api_key)
    loadings = dict()
    for tile in tiles:
        print(f"Loading images for tile: {tile}...")
        loaded = loader.download(product_type,
                                 [tile],
                                 start_date=start_date,
                                 end_date=end_date,
                                 output_dir=output_dir,                       
                                 bands=BANDS,
                                constraints=CONSTRAINTS)
        
        print(f"Loading images for tile {tile} finished")
        loadings[tile] = loaded
    
    # tile_folders = dict()
    # for tile, tile_paths in loadings.items():
    #    tile_folders[tile] = {str(Path(tile_path[0]).parent) for tile_path in tile_paths}
    return loadings

In [33]:
loadings = load_images(API_KEY, sentinel_tiles.Name.values, start, end, LOAD_DIR, PRODUCT_TYPE)

Loading images for tile: 36UXA...
Loading images for tile 36UXA finished


In [34]:
loadings

{'36UXA': [('/home/jovyan/work/satellite_imagery/S2A_MSIL2A_20190825T083601_N0213_R064_T36UXA_20190825T112543/T36UXA_20190825T083601_TCI_10m.jp2',
   'L2/tiles/36/U/XA/S2A_MSIL2A_20190825T083601_N0213_R064_T36UXA_20190825T112543.SAFE/GRANULE/L2A_T36UXA_A021797_20190825T083616/IMG_DATA/R10m/T36UXA_20190825T083601_TCI_10m.jp2'),
  ('/home/jovyan/work/satellite_imagery/S2A_MSIL2A_20190825T083601_N0213_R064_T36UXA_20190825T112543/MTD_TL.xml',
   'L2/tiles/36/U/XA/S2A_MSIL2A_20190825T083601_N0213_R064_T36UXA_20190825T112543.SAFE/GRANULE/L2A_T36UXA_A021797_20190825T083616/MTD_TL.xml'),
  ('/home/jovyan/work/satellite_imagery/S2A_MSIL2A_20190828T084601_N0213_R107_T36UXA_20190828T131555/T36UXA_20190828T084601_TCI_10m.jp2',
   'L2/tiles/36/U/XA/S2A_MSIL2A_20190828T084601_N0213_R107_T36UXA_20190828T131555.SAFE/GRANULE/L2A_T36UXA_A021840_20190828T084904/IMG_DATA/R10m/T36UXA_20190828T084601_TCI_10m.jp2'),
  ('/home/jovyan/work/satellite_imagery/S2A_MSIL2A_20190828T084601_N0213_R107_T36UXA_20190828

In [35]:
def filter_date(loadings):
    def _find_last_date(folders):        
        dates = list()
        for folder in folders:        
            search = re.search(r"_(\d+)T\d+_", str(folder))
            date = search.group(1)
            date = datetime.strptime(date, '%Y%m%d')
            dates.append(date)    
        last_date = max(dates)
        last_date = datetime.strftime(last_date, '%Y%m%d')
        return last_date
    
    filtered = dict()
    for tile, items in loadings.items():
        try:
            last_date = _find_last_date(items)
            for file, _ in items:
                if "TCI_10m.jp2" in file and last_date in file:
                    filtered[tile] = file
        except Exception as ex:
            print(f"Error for {tile}: {str(ex)}")
    return filtered

In [36]:
filtered = filter_date(loadings)
filtered

{'36UXA': '/home/jovyan/work/satellite_imagery/S2A_MSIL2A_20190828T084601_N0213_R107_T36UXA_20190828T131555/T36UXA_20190828T084601_TCI_10m.jp2'}

In [37]:
if not filtered:
    raise ValueError("Images not loaded. Change dates or constraints")

### Prepair Sentinel2 images for AOI

In [38]:
with tempfile.TemporaryDirectory(dir=WORKDIR) as tmpdirname:        
    print(f"Сreated temporary directory for calculations: {tmpdirname}")    
    files_to_mosaic = list()
    
    aoi_name = Path(aoi_path).stem
    for row in sentinel_tiles.itertuples():
        
        name = f"{aoi_name}_{row.Name}"
        
        input_path = filtered[row.Name]        
        output_path = os.path.join(WORKDIR, f"{name}_cropped.tif")
        
        crop(input_path, output_path, row.geometry)
        files_to_mosaic.append(output_path)
    
    create_mosaic(files_to_mosaic, WORKDIR, name)
    print(f"Calculations finished")

Сreated temporary directory for calculations: /home/jovyan/work/notebooks/example/planet_vs_sentinel/tmpz4qilpq5
Calculations finished


#### Select clearcuts from original file provided by V. Kharmtsov

In [150]:
aoi_name = Path(aoi_path).stem
clearcuts = gpd.read_file("original_clearcuts.geojson")

In [151]:
clearcuts.tail(2)

,img_date,tileID,geometry
210,2019-05-15,20190515_065304_0f2d,"POLYGON ((35.10101 50.13608, 35.10149 50.13608..."
211,2019-05-15,20190515_065304_0f2d,"POLYGON ((35.10310 50.13460, 35.10333 50.13446..."


In [152]:
aoi

,id,geometry
0,1,"MULTIPOLYGON (((35.27794 50.15967, 35.22820 50..."


In [153]:
clearcuts = clearcuts.loc[(clearcuts.img_date >= start_date) & 
                          (clearcuts.img_date <= end_date)]
clearcuts.tail(2)

,img_date,tileID,geometry
189,2019-08-25,20190825_083010_43_105d,"POLYGON ((35.33780 50.10290, 35.33770 50.10298..."
190,2019-08-30,20190830_081707_1021,"POLYGON ((35.41443 50.12672, 35.41444 50.12662..."


In [154]:
aoi_clearcuts = gpd.overlay(clearcuts, aoi, how="intersection") 
aoi_clearcuts.tail(2)

,tileID,img_date,id,geometry


In [157]:
if not aoi_clearcuts.empty:
    aoi_clearcuts.to_file(os.path.join(WORKDIR, f"{aoi_name}_clearcuts.geojson"), driver='GeoJSON')